In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import base
import gym

In [3]:
from src.models import build_deep_mind_model
from src.agents import DeepQAgent

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Environment

In [4]:
env = gym.make('SpaceInvaders-v0')

In [5]:
num_actions = env.action_space.n
num_actions

6

# Model

In [6]:
model = build_deep_mind_model(
    image_size=(84, 84),
    num_frames=4,
    num_actions=num_actions
)
model

# Agent

In [7]:
agent = DeepQAgent(model, env)
agent

DeepQAgent(
    model=<keras.models.Sequential object at 0x7facc8b74550>,
    env=<TimeLimit<AtariEnv<SpaceInvaders-v0>>>,
    learning_rate=0.001,
    discount_factor=0.99,
    exploration_rate=0.8,
    exploration_decay=0.99,
    episodes=1000
)

In [8]:
agent.run()

ValueError: cannot reshape array of size 100800 into shape (1,84,84,4)